<a href="https://colab.research.google.com/github/gmflau/GenAI/blob/main/colab/Install_Ollama_in_a_Colab_machine_and_set_up_pyngrok_for_local_access.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Run any AI model remotely for free on google colab with Ollama

Instructions based on:    
this youtube => https://www.youtube.com/watch?v=Qa1h7ygwQq8
this Colab => https://github.com/ollama/ollama/blob/5687f1a0cfa3d2408bfcb04f4342f657f6dada58/examples/jupyter-notebook/ollama.ipynb

In [1]:
# Download and install ollama to the system
!curl https://ollama.ai/install.sh | sh

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0>>> Downloading ollama...
100  8575    0  8575    0     0  24097      0 --:--:-- --:--:-- --:--:-- 24087
############################################################################################# 100.0%
>>> Installing ollama to /usr/local/bin...
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [ ]:
!pip install aiohttp pyngrok

from google.colab import userdata
import os
import asyncio

# Set LD_LIBRARY_PATH so the system NVIDIA library
os.environ.update({'LD_LIBRARY_PATH': '/usr/lib64-nvidia'})

async def run_process(cmd):
  print('>>> starting', *cmd)
  p = await asyncio.subprocess.create_subprocess_exec(
      *cmd,
      stdout=asyncio.subprocess.PIPE,
      stderr=asyncio.subprocess.PIPE,
  )

  async def pipe(lines):
    async for line in lines:
      print(line.strip().decode('utf-8'))

  await asyncio.gather(
      pipe(p.stdout),
      pipe(p.stderr),
  )

#register an account at ngrok.com and create an authtoken and place it here
await asyncio.gather(
    run_process(['ngrok', 'config', 'add-authtoken', userdata.get('ngrok_token')])
)

await asyncio.gather(
    run_process(['ollama', 'serve']),
    run_process(['ngrok', 'http', '--log', 'stderr', '11434']),
)

>>> starting ngrok config add-authtoken 2d8mMd25gcg0UK7r6lpx6lPpGND_6B4VEKFWprVk9CpkaWfSw
Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
>>> starting ollama serve
>>> starting ngrok http --log stderr 11434
Couldn't find '/root/.ollama/id_ed25519'. Generating new private key.
Your new public key is:

ssh-ed25519 AAAAC3NzaC1lZDI1NTE5AAAAIFTFsZ8PwcVabqk2IILE0CrczFLQheCE60LspeT/AtDh

time=2024-03-03T01:04:03.120Z level=INFO source=images.go:710 msg="total blobs: 0"
time=2024-03-03T01:04:03.121Z level=INFO source=images.go:717 msg="total unused blobs removed: 0"
time=2024-03-03T01:04:03.121Z level=INFO source=routes.go:1019 msg="Listening on 127.0.0.1:11434 (version 0.1.27)"
time=2024-03-03T01:04:03.121Z level=INFO source=payload_common.go:107 msg="Extracting dynamic libraries..."
t=2024-03-03T01:04:03+0000 lvl=info msg="no configuration paths supplied"
t=2024-03-03T01:04:03+0000 lvl=info msg="using configuration at default config path" path=/root/.config/ngrok/ngrok.y

The previous cell starts two processes, ollama and ngrok. The log output will show a line like the following which describes the external address.
    
t=2023-11-12T22:55:56+0000 lvl=info msg="started tunnel" obj=tunnels name=command_line addr=http://localhost:11434 url=https://{your-id}.ngrok.io   
The external address in this case is https://{your id}.ngrok.io which can be passed into OLLAMA_HOST to access this instance.

export OLLAMA_HOST=https://{your-id}.ngrok.io   
ollama list   
ollama run gemma   
ollama run mistral